# 1. Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Definições

In [84]:
colunas_redundantes = [
    'unidade_cadastro',
    'cod_regulacao',
    'cod_unidade_origem',
    'tipo_regulacao',
    'data_cadastro',
    'data_lista_reg',
    'data_vaga_reg',
    'data_admissao_reg',
    'data_lista_hospital',
    'data_vaga_hospital',
    'data_internacao',
    'data_cancel_solicitacao',
    'data_transferencia',
    'gravidez',
    'tipo_internacao_de',
    'data_expiracao',
    'cod_transf_alta',
    'nome_unidade_solicitacao',
    'nome_unidade_destino',
    'via_internacao',
    'idade',
    'faixa_etaria',
    'diff_in_sec',
    'diff_in_min',
    'diff_in_hour',
    'diff_in_day',
    'diff_delay_Rank_vaga_internacao_1',
    'Count',
    'total_repeat_cod',
    'total_repeat',
    'sum_los_hour',
    'sum_outside_hour',
    'cumsum',
    'cumsum.1',
    'cumsum.1(start)',
    'cod_paciente.1',
    'start',
    'stop',
    'uf',
    'municipio',
    'idade',
    'idade_aprox',
    'status_internacao',
    'tipo_alta',
    'tipo_internacao',
    'tempo_espera_internacao',
    'tempo_espera_aceite',
    'cid10_diagnostico_alta',
    'cid10_seg_diagnostico_alta',
    'grupo_diagnostico_alta',
    'tempo_internacao',
    'event',
    'cid10_seg_diagnostico',
    'data_solicitacao'
]

PATH_INPUT_DATASET = '../data/raw/total dataset with los and outsid time.csv'
PATH_MAPPING_PROFISSAO = "../data/raw/profissoes_mapping.txt"
PATH_OUTPUT_PROCESSED_DATASET = "../data/processed/"
PATH_DE_PARA_CID = '../data/raw/CID_DEPARA.csv'


estado_civil_mapping = {
    'SOLTEIRO': 'solteiro',
    'CASADO': 'casado',
    'NÃO INFORMADO': 'nao informado',
    'viasvo': 'nao informado',
    'SEPARADO': 'solteiro',
    'DIVORCIADO': 'solteiro'
}

etnia_mapping = {
    'BRANCO': 'branco',
    'PARDO': 'pardo',
    'NEGRO': 'preto',
    'NÃO INFORMADO': 'nao informado',
    'AMARELO': 'amarelo',
}

arranjo_domiciliar_mapping = {
    'NÃO INFORMADO': 'nao informado',
    'SOZINHO': 'sozinho',
    'COM COMPANHEIRO ( A)': 'acompanhado',
    'COM FILHOS(AS)': 'acompanhado',
    'COM PAI/MÃE': 'acompanhado',
    'COM OUTROS FAMILIARES': 'acompanhado',
    'OUTRO': 'nao informado',
    'CONHECIDOS/AMIGOS': 'acompanhado'
}

## Funções de apoio

In [103]:
def ler_dataset(path, **kwargs):
    
    return pd.read_csv(path, **kwargs)

def pegar_mapping_profissao(path):
    with open(path, "r") as arquivo:
        
        return eval(arquivo.read())

def remover_colunas_redundantes(df, cols_to_drop):

    return df.drop(cols_to_drop, axis=1)

def tratar_colunas_data(df):
    colunas_simples_tratamento = ['data_internacao_real', 'data_alta']
    colunas_tratamento_regex = ['data_nascimento']
    padrao_data_regex = "(\d{1,2}/\d{1,2}/\d{4})"

    for coluna in colunas_simples_tratamento:
        df[coluna] = pd.to_datetime(df[coluna], format='%m/%d/%Y %H:%M', errors='raise')

    for coluna in colunas_tratamento_regex:
        df[coluna] = pd.to_datetime(df[coluna].str.extract(padrao_data_regex).to_numpy().flat, format='%m/%d/%Y')

    return df

def criar_situacao_profissao(df):
    profissao_mapping = pegar_mapping_profissao(PATH_MAPPING_PROFISSAO)
    
    df['profissao_ocupacao'] = df['profissao_ocupacao'].str.lower()
    df['situacao_profissao'] = df['profissao_ocupacao'].map(lambda x: profissao_mapping.get(x, 'nao informado'))
    df = df.drop('profissao_ocupacao', axis=1)

    return df

def tratar_estado_civil(df):
    df['estado_civil'] = df['estado_civil'].map(estado_civil_mapping)

    return df

def tratar_etnia(df):
    df['etnia'] = df['etnia'].map(etnia_mapping)

    return df

def tratar_arranjo_domiciliar(df):
    df['arranjo_domiciliar'] = df['arranjo_domiciliar'].map(arranjo_domiciliar_mapping)

    return df

def tratar_cid_diagnostico(df):
    cid_df = ler_dataset(PATH_DE_PARA_CID, sep=';', encoding='latin-1')

    df['cid10_diagnostico'] = df['cid10_diagnostico'].str.replace('.', '', regex=False).str.upper()
    df = (
        df
        .merge(cid_df[['SUBCATEGORIA', 'DESCR_CAT', 'GRUPO', 'CAPITULO']], left_on='cid10_diagnostico', right_on='SUBCATEGORIA', how='left')
        .drop('SUBCATEGORIA', axis=1)
        .rename(columns={'DESCR_CAT': 'cid_diagnostico_descricao', 'GRUPO': 'cid_diagnostico_grupo', 'CAPITULO': 'cid_diagnostico_capitulo'})
        .fillna({'cid_diagnostico_descricao': 'nao informado', 'cid_diagnostico_grupo': 'nao informado', 'cid_diagnostico_capitulo': 'nao informado'})
    )

    return df

def tratar_colunas_binarias(df):
    cols_binarias = [
        'traumatismo',
        'prob_respiratorios',
        'avc',
        'convulsao',
        'has',
        'diabetes',
        'doenca_infecto'
    ]

    for col in cols_binarias:
        df[col] = df[col].apply(lambda x: 1 if x == 'SIM' else 0)

    return df

def remover_nulos(df):
    return df.dropna()

# 2. Dataprep

In [104]:
df = ler_dataset(PATH_INPUT_DATASET, encoding="latin1")

transformed_df = (
    df
    .pipe(remover_colunas_redundantes, colunas_redundantes)
    .pipe(tratar_colunas_data)
    .pipe(criar_situacao_profissao)
    .pipe(tratar_estado_civil)
    .pipe(tratar_etnia)
    .pipe(tratar_arranjo_domiciliar)
    .pipe(tratar_cid_diagnostico)
    .pipe(tratar_colunas_binarias)
    .pipe(remover_nulos)
)

transformed_df

,data_internacao_real,sexo,data_nascimento,cod_paciente,estado_civil,etnia,arranjo_domiciliar,cid10_diagnostico,traumatismo,prob_respiratorios,avc,convulsao,has,diabetes,doenca_infecto,data_alta,situacao_profissao,cid_diagnostico_descricao,cid_diagnostico_grupo,cid_diagnostico_capitulo
2,2013-04-12 18:30:00,F,1971-02-10,40,solteiro,branco,nao informado,F142,0,0,0,0,0,0,0,2013-12-06 17:00:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais
3,2014-11-12 10:06:00,F,1971-02-10,40,solteiro,branco,nao informado,F142,0,0,0,0,0,0,0,2014-11-20 09:33:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais
4,2015-01-15 12:50:00,F,1971-02-10,40,solteiro,branco,nao informado,F145,0,0,0,0,0,0,0,2015-01-27 08:09:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais
5,2015-04-23 17:37:00,F,1971-02-10,40,solteiro,branco,nao informado,F145,0,0,0,0,0,0,0,2015-04-27 20:04:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais
6,2015-07-02 11:38:00,F,1971-02-10,40,solteiro,branco,nao informado,F603,0,0,0,0,0,0,0,2016-02-23 13:31:00,inativa,Transtornos específicos da personalidade,Transtornos da personalidade e do comportament...,Transtornos mentais e comportamentais
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8750,2017-12-27 14:27:00,F,1998-12-27,11064,nao informado,nao informado,nao informado,F192,0,0,0,0,0,0,0,2017-12-30 20:12:00,nao informada,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais
8751,2017-12-27 14:31:00,F,1961-05-22,11065,nao informado,nao informado,acompanhado,F322,0,0,0,0,1,0,0,2017-12-29 19:12:00,nao informada,Episódios depressivos,Transtornos do humor [afetivos],Transtornos mentais e comportamentais
8752,2017-12-30 12:48:00,F,2002-08-08,11070,nao informado,nao informado,nao informado,F322,0,0,0,0,0,0,0,2018-01-03 14:01:00,nao informada,Episódios depressivos,Transtornos do humor [afetivos],Transtornos mentais e comportamentais
8753,2017-12-29 18:30:00,F,1951-03-03,11075,nao informado,branco,nao informado,F445,0,0,0,0,1,1,0,2018-01-23 07:54:00,inativa,Transtornos dissociativos [de conversão],"Transtornos neuróticos, transtornos relacionad...",Transtornos mentais e comportamentais


In [105]:
transformed_df.columns

Index(['data_internacao_real', 'sexo', 'data_nascimento', 'cod_paciente',
       'estado_civil', 'etnia', 'arranjo_domiciliar', 'cid10_diagnostico',
       'traumatismo', 'prob_respiratorios', 'avc', 'convulsao', 'has',
       'diabetes', 'doenca_infecto', 'data_alta', 'situacao_profissao',
       'cid_diagnostico_descricao', 'cid_diagnostico_grupo',
       'cid_diagnostico_capitulo'],
      dtype='object')

In [106]:
transformed_df.shape[0]

8000

In [107]:
# Colunas a serem criadas
# ano e mes da internacao
# idade no momento da internação (nascimento - data internacao)
# tempo internação (data_alta - data_internacao_real)
# 

data_internacao_real         0
sexo                         0
data_nascimento              0
cod_paciente                 0
estado_civil                 0
etnia                        0
arranjo_domiciliar           0
cid10_diagnostico            0
traumatismo                  0
prob_respiratorios           0
avc                          0
convulsao                    0
has                          0
diabetes                     0
doenca_infecto               0
data_alta                    0
situacao_profissao           0
cid_diagnostico_descricao    0
cid_diagnostico_grupo        0
cid_diagnostico_capitulo     0
dtype: int64

In [121]:
transformed_df['ano_data_internacao'] = transformed_df['data_internacao_real'].dt.year
transformed_df['mes_data_internacao'] = transformed_df['data_internacao_real'].dt.month
# transformed_df['idade_data_internacao'] = (transformed_df['data_internacao_real'] - transformed_df['data_nascimento']) // 365

In [122]:
transformed_df

,data_internacao_real,sexo,data_nascimento,cod_paciente,estado_civil,etnia,arranjo_domiciliar,cid10_diagnostico,traumatismo,prob_respiratorios,...,diabetes,doenca_infecto,data_alta,situacao_profissao,cid_diagnostico_descricao,cid_diagnostico_grupo,cid_diagnostico_capitulo,ano_data_internacao,mes_data_internacao,idade_data_internacao
2,2013-04-12 18:30:00,F,1971-02-10,40,solteiro,branco,nao informado,F142,0,0,...,0,0,2013-12-06 17:00:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais,2013,4,42 days 04:47:05.753424657
3,2014-11-12 10:06:00,F,1971-02-10,40,solteiro,branco,nao informado,F142,0,0,...,0,0,2014-11-20 09:33:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais,2014,11,43 days 18:49:59.342465753
4,2015-01-15 12:50:00,F,1971-02-10,40,solteiro,branco,nao informado,F145,0,0,...,0,0,2015-01-27 08:09:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais,2015,1,43 days 23:02:55.890410958
5,2015-04-23 17:37:00,F,1971-02-10,40,solteiro,branco,nao informado,F145,0,0,...,0,0,2015-04-27 20:04:00,inativa,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais,2015,4,44 days 05:30:20.876712328
6,2015-07-02 11:38:00,F,1971-02-10,40,solteiro,branco,nao informado,F603,0,0,...,0,0,2016-02-23 13:31:00,inativa,Transtornos específicos da personalidade,Transtornos da personalidade e do comportament...,Transtornos mentais e comportamentais,2015,7,44 days 10:05:31.726027397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8750,2017-12-27 14:27:00,F,1998-12-27,11064,nao informado,nao informado,nao informado,F192,0,0,...,0,0,2017-12-30 20:12:00,nao informada,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais,2017,12,19 days 00:22:06.082191780
8751,2017-12-27 14:31:00,F,1961-05-22,11065,nao informado,nao informado,acompanhado,F322,0,0,...,0,0,2017-12-29 19:12:00,nao informada,Episódios depressivos,Transtornos do humor [afetivos],Transtornos mentais e comportamentais,2017,12,56 days 15:21:37.150684931
8752,2017-12-30 12:48:00,F,2002-08-08,11070,nao informado,nao informado,nao informado,F322,0,0,...,0,0,2018-01-03 14:01:00,nao informada,Episódios depressivos,Transtornos do humor [afetivos],Transtornos mentais e comportamentais,2017,12,15 days 09:45:59.671232876
8753,2017-12-29 18:30:00,F,1951-03-03,11075,nao informado,branco,nao informado,F445,0,0,...,1,0,2018-01-23 07:54:00,inativa,Transtornos dissociativos [de conversão],"Transtornos neuróticos, transtornos relacionad...",Transtornos mentais e comportamentais,2017,12,66 days 20:57:36.986301369
